# Import the modules 

In [1]:
# import the general modules
from optlang.symbolics import Zero
import cobra
import cplex
import sys

# import kbase
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'I6WL7MAW5Z4QO3OQ62YUR6HJM6RAUFHE'
kbase = cobrakbase.KBaseAPI(token)

# define the example individual model and associated API media package
#model = kbase.get_from_ws('e_coli_core', 94253)
model = kbase.get_from_ws('e_coli_core.kb', 94253)
#model = kbase.get_from_ws('E_iAH991V2', 94253)
model.solver = 'optlang-cplex'

# local ModelSEED database path
modelseed_db_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'

cobrakbase 0.2.8


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# RevBin package test

The RevBin package prevents directional fluxes through reactions that have net zero flux. This is exemplified in the following demonstration, where the forward progression of the reversible ethanol transport reaction (rxn08427_c0) is optimized despite lacking the ability to have net flux. The unconstrained simulation possesses a maximal objective value, while the RevBin-constrained simulation possesses an objective value of zero.    

## Unconstrained simulation

In [2]:
model = kbase.get_from_ws('e_coli_core.kb', 94253)

with model:
    # set the objective function
    model.objective = model.reactions.rxn08427_c0.forward_variable 
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## RevBin-constrained simulation

### Constrain and execute the model

In [3]:
from modelseedpy.fbapkg import revbinpkg

# apply the RevBin constraints 
revbin = revbinpkg.RevBinPkg(model)
revbin.build_package()

with model:
    # set the objective function
    model.objective = model.reactions.rxn08427_c0.forward_variable 
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,100,0,0.00%
cpd00020_e0,EX_cpd00020_e0,800,3,54.55%
cpd00363_e0,EX_cpd00363_e0,1000,2,45.45%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00001_e0,EX_cpd00001_e0,-200,0,0.00%
cpd00071_e0,EX_cpd00071_e0,-1000,2,45.45%
cpd00221_e0,EX_cpd00221_e0,-800,3,54.55%


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# SimpleThermo package test

The SimpleThermo package prevents flux through thermodynamically infeasible reactions. This is exemplified in the following demonstrations. 

The first demonstration executes the model in an empty media, which is an infeasible environment for growth. The unconstrained model grows, while the SimpleThermo-constrained model does not grow. 
The second demonstration optimizes the model for an internal loop (FRD7_c0) in the *E. coli* core model. The unconstrained model possesses a maximal objective value, while the SimpleThermo-constrained model possesses an objective value of zero.   

## Unconstrained simulation

### Execute the model with conventional objective

In [4]:
model = kbase.get_from_ws('e_coli_core.kb', 94253)

with model:
    # edit the model media
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)

    # execute the model
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the model with internal loop objective

In [5]:
with model:
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## SimpleThermo-constrained simulation

### Constrain the model

In [6]:
# ensure that all charges are integers
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
        
# apply the simple thermodynamic constraints 
from modelseedpy.fbapkg import simplethermopkg
stp = simplethermopkg.SimpleThermoPkg(model)
stp.build_package(stp.parameters)

# export the constrained LP file 
with open('SimpleThermo.lp', 'w') as out:
    out.write(str(model.solver))

### Execute the model in empty media

In [7]:
with model:
    # edit the model media
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)

    # execute the model
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the model with an optimized internal flux

In [8]:
with model:
    # execute the simulation
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# FullThermo package test

The FullThermo package prevents flux through thermodynamically infeasible reactions, with greater specificity than the SimpleThermo package. This functionality is exemplified with a set of simulation conditions that appropriately grow in the unconstrained model yet prevent growth in the constrained model. These conditions presumably create a thermodynamically infeasible problem that prevents the constrained model yet allows the unconstrained model to grow. 

## Unconstrained simulation

### Execute the model with an optimized internal flux

In [13]:
model = kbase.get_from_ws('e_coli_core.kb', 94253)
with model:
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the unconstrained simulation in infeasible media 

In [14]:
with model:
    # edit the model medium
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)

    # execute the model
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## FullThermo-constrained simulation

### Constrain the model

In [15]:
from modelseedpy.fbapkg import fullthermopkg

# implement FullThermo with dgbin constraints
ftp = fullthermopkg.FullThermoPkg(model)
ftp.build_package({'modelseed_path':modelseed_db_path})

# export the constrained LP file 
with open('FullThermo.lp', 'w') as out:
    out.write(str(model.solver))

### Execute the model with an optimized internal flux

In [16]:
with model:
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the constrained model in infeasible media

In [17]:
with model:
    # define the simulation medium
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)
    
    # print the results
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Brainstorming

In [ ]:
for reaction in model.reactions:
    print(reaction.id)

In [ ]:
objective_coefficient = {}
objective_coefficient[model.reactions.rxn08427_c0.forward_variable] = 1

# define the maximum progression
model.objective = model.problem.Objective(Zero, direction='max')
model.objective.set_linear_coefficients(objective_coefficient)

# execute the model
model.summary()

In [ ]:
forv = model.reactions.rxn08427_c0.forward_variable 
print(forv)
dir(model.reactions.rxn08427_c0)